# DIABETES DIAGNOSIS
DATA:
 - Pregnancies
 - Glucose
 - BloodPressure
 - Skin THickness
 - INsulin
 - BMI
 - DiabetesPedigreeFunction
 - AGe
 - Outcome

IMPORTING IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn.metrics
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data= pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
data

In [ ]:
data.describe()

In [ ]:
col = data.columns[:- 1]
col

In [ ]:
data.info()

As we know 0's in columns are same as nan but can interfere with code performance,
Replace all the 0's with Nan 

In [ ]:
data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = data[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)


In [ ]:
data.describe()

In [ ]:
p = data.hist(figsize = (20,20))

Fillings Nan values with Mean, Meadian as per the skewness and presence of outliers,
in CASE OF OUTLIERS, WE USE mEDIANS,FOR CATEGROICAL WE USE mode AND WITHOUT OUTLIER CONTINOUS VLAUES WE USE mean

In [ ]:
data['Glucose'].fillna(data['Glucose'].mean(), inplace = True)
data['BloodPressure'].fillna(data['BloodPressure'].mean(), inplace = True)
data['SkinThickness'].fillna(data['SkinThickness'].median(), inplace = True)
data['Insulin'].fillna(data['Insulin'].median(), inplace = True)
data['BMI'].fillna(data['BMI'].median(), inplace = True)

In [ ]:
p = data.hist(figsize=(10,10))

# EDA

In [ ]:
sb.heatmap(data.isnull())

In [ ]:
col = data.columns[:-1]

In [ ]:
col

checking outliers

In [ ]:
plt.figure(figsize=(20,18))
for i in range(len(col)):
    plt.subplot(4,2,i+1)
    data[col[i]].plot(kind='box')
    
    
    

In [ ]:
#CHECKING DIstribution of columns

In [ ]:
plt.figure(figsize=(20,18))
for i in range(len(col)):
    plt.subplot(4,2,i+1)
    data[col[i]].plot(kind='hist')
    
    
    

OUTLIER TREATMENT, but as no of outliers are high in number, Removing outliers not an option,instead scaling data

In [ ]:

Q1 = data.quantile(.25)
Q3 = data.quantile(.75)
IQR = Q3 - Q1
count = ( ( data.iloc[:] > ( Q3 + 1.5*IQR )) | ( data.iloc[:] <  (Q1 - 1.5*IQR)) ).sum(axis=0)

In [ ]:
count

In [ ]:
data = data[~( ( data.iloc[:] < (Q1 - 1.5*IQR ) ) |( data.iloc[:] > (Q3 + 1.5*IQR ) )).any(axis=1)]
count = ( ( data.iloc[:] > ( Q3 + 1.5*IQR )) | ( data.iloc[:] <  (Q1 - 1.5*IQR)) ).sum(axis=0)
count

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

In [ ]:
scale.fit(data.drop('Outcome',axis=1))

In [ ]:
features = scale.fit_transform(data.drop('Outcome',axis=1))


In [ ]:
new_data = pd.DataFrame(features,columns=col)
new_data

In [ ]:
sb.heatmap(data.corr(),annot=True)

In [ ]:
cor = new_data.corr()

In [ ]:
cor

In [ ]:
sb.scatterplot('SkinThickness','BMI',data=data,hue='Outcome')

In [ ]:
sb.pairplot(data)

In [ ]:
plt.figure(figsize=(20,18))
for i in range(len(col)):
    plt.subplot(4,2,i+1)
    new_data[col[i]].plot(kind='hist')
    

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(new_data,data['Outcome'],test_size=.28,random_state = 45)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn =KNeighborsClassifier(n_neighbors=1)

In [ ]:
knn.fit(x_train,y_train)

In [ ]:
error = []
for i in range(1,40):
    knn_test = KNeighborsClassifier(n_neighbors=i)
    knn_test.fit(x_train,y_train)
    pred_i = knn_test.predict(x_test)
    error.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(15,10))
plt.title('ErrorRate VS K-Value')
plt.plot(range(1,40),error,marker='*',linestyle='dashed')


In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
knn_final = KNeighborsClassifier(n_neighbors=13)
knn_final.fit(x_train,y_train)
pred = knn_final.predict(x_test)
accuracy_score(y_test,pred)

In [ ]:
from sklearn.linear_model import RidgeClassifier,ElasticNet,LogisticRegression

Checking Performance with Other MOdels

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc = SVC(kernel='linear',C=1)

In [ ]:
svc.fit(x_train,y_train)
pred_l = svc.predict(x_test)
accuracy_score(y_test,pred_l)

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
y_pred_proba = knn_final.predict_proba(x_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

In [ ]:
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr,tpr, label='Knn')
plt.xlabel('1-specificity')
plt.ylabel('sensitivity')
plt.title('Knn(n_neighbors=11) ROC curve')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred_proba)

# Hyper Parameter optimization
Grid search is an approach to hyperparameter tuning that will methodically build and evaluate a model for each combination of algorithm parameters specified in a grid.

In [ ]:
from sklearn.model_selection import GridSearchCV
#In case of classifier like knn the parameter to be tuned is n_neighbors
param_grid = {'n_neighbors':np.arange(1,50)}
knn = KNeighborsClassifier()
knn_cv= GridSearchCV(knn,param_grid,cv=5)
knn_cv.fit(new_data,data['Outcome'])

print("Best Score:" + str(knn_cv.best_score_))
print("Best Parameters: " + str(knn_cv.best_params_))